In [1]:
import os
import json
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertModel, BertConfig, BertForSequenceClassification, AdamW
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
import fasttext

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/transformers/utils/generic.py:311: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/transformers/utils/generic.py:311: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(


In [2]:
val_path= './val_file.json' #"/kaggle/input/erc-and-efr/val_file.json"

with open(val_path) as f:
    val_list = json.load(f)



val_list,test_list=train_test_split(val_list, test_size=0.1, random_state=42)


# create test.json
test_data=[]
for item in test_list:
    episode = item['episode']
    speakers = item['speakers']
    emotions = item['emotions']
    utterances = item['utterances']
    triggers = item['triggers']
    item_dict = {
        'episode': episode,
        'speakers': speakers,
        'emotions': emotions,
        'utterances': utterances,
        'triggers': triggers
    }
    test_data.append(item_dict)

json_string = json.dumps(test_data)
with open('./test_file.json', 'w') as f:
    f.write(json_string)

label_map = {'anger':0, 'surprise':1, 'sadness':2, 'neutral':3, 'fear':4, 'joy':5, 'disgust':6}  # Map emotions to labels

In [3]:
def preprocess_for_ERC_Model1(dialogues,emotions,tokenizer,max_len):
     global label_map
     input_ids = []
     attention_masks=[]
     for i in range(len(dialogues)):
          for j in range(len(dialogues[i])):
              window_text = ' [SEP] '.join(dialogues[i][max(0,j-4):j+1])  # Concatenate the current utterance with previous 5 utterances if possible
              encoded_input = tokenizer(window_text, return_tensors='pt',padding='max_length', truncation=True, max_length=max_len)
              input_ids.append(encoded_input['input_ids'].flatten())
              attention_masks.append(encoded_input['attention_mask'].flatten())

     labels = torch.tensor([label_map[emotion] for emotion in emotions])
     return input_ids,attention_masks,labels


def preprocess_for_ERC_Model2(dialogues,emotions):
    
    global label_map 
    # FastText model
    fasttext_model = fasttext.load_model('./Embeddings/cc.en.300.bin') #('/kaggle/input/embeddings/cc.en.300.bin')
    sentence_embeddings = []
    for i in range(len(dialogues)):
        for j in range(len(dialogues[i])):
            sentence=dialogues[i][j] 
            word_embeddings = [fasttext_model.get_word_vector(word) for word in sentence.split()]
            sentence_embedding = np.mean(word_embeddings, axis=0)
            sentence_embeddings.append(torch.tensor(sentence_embedding))
    labels = torch.tensor([label_map[emotion] for emotion in emotions])
    sentence_embeddings=torch.stack(sentence_embeddings)
    return sentence_embeddings,None,labels
  

def preprocess_for_EFR_Model3(dialogues,speakers,triggers,tokenizer,max_len):
     global label_map
     input_ids = []
     attention_masks=[]
     label_triggers=[]
     lens=[]

     for i in range(len(dialogues)):
          last_text_by_a_speaker=-1
          previous_flipped_emotion_text_by_same_speaker=-1
          speaker=speakers[i][-1]
          for j in range(len(dialogues[i])-1,-1,-1):
              if speakers[i][j]==speaker:
                  previous_flipped_emotion_text_by_same_speaker=j
                  break

          curr_input_ids=[]
          curr_attention_masks=[]

          for j in range(len(dialogues[i])):
              window_text = dialogues[i][j]  # Concatenate the current utterance with previous 5 utterances if possible
              encoded_input = tokenizer(window_text, return_tensors='pt',padding='max_length', truncation=True, max_length=max_len)
              lens.append(len(encoded_input['input_ids'].flatten()))
              curr_input_ids.append(encoded_input['input_ids'].flatten())
              curr_attention_masks.append(encoded_input['attention_mask'].flatten())


          for j in range(len(dialogues[i])):
              curr_input_ids[j]=torch.cat((curr_input_ids[j],curr_input_ids[last_text_by_a_speaker],curr_input_ids[previous_flipped_emotion_text_by_same_speaker]),dim=0)
              curr_attention_masks[j]=torch.cat((curr_attention_masks[j],curr_attention_masks[last_text_by_a_speaker],curr_attention_masks[previous_flipped_emotion_text_by_same_speaker]),dim=0)

          input_ids.extend(curr_input_ids)
          attention_masks.extend(curr_attention_masks)
          
          try:
              k=sum(triggers[i])
          except:
                 for a in range(len(triggers[i])):
                     if triggers[i][a]==None: triggers[i][a]=0.0

          label_triggers.extend(triggers[i]) # skip those instances where triggers has None values
     return input_ids,attention_masks,label_triggers



def preprocess_for_EFR_Model4(dialogues,emotions,speakers,triggers):
    global label_map 
    # FastText model
    fasttext_model = fasttext.load_model('./Embeddings/cc.en.300.bin') #('/kaggle/input/embeddings/cc.en.300.bin')
    sentence_embeddings = []
    label_triggers=[]
    for i in range(len(dialogues)):
        last_text_by_a_speaker=-1
        previous_flipped_emotion_text_by_same_speaker=-1
        speaker=speakers[i][-1]
        for j in range(len(dialogues[i])-1,-1,-1):
            if speakers[i][j]==speaker:
                previous_flipped_emotion_text_by_same_speaker=j
                break

        curr_sentence_embeddings=[]
          
        for j in range(len(dialogues[i])):
            sentence=dialogues[i][j] 
            word_embeddings = [fasttext_model.get_word_vector(word) for word in sentence.split()]
            sentence_embedding = np.mean(word_embeddings, axis=0)
            curr_sentence_embeddings.append(torch.tensor(sentence_embedding))
            
            
        for j in range(len(dialogues[i])):
            curr_sentence_embeddings[j]=torch.cat((curr_sentence_embeddings[j],curr_sentence_embeddings[last_text_by_a_speaker],curr_sentence_embeddings[previous_flipped_emotion_text_by_same_speaker]),dim=0)
            

        sentence_embeddings.extend(curr_sentence_embeddings)
        
        try:
            k=sum(triggers[i])
        except:
                for a in range(len(triggers[i])):
                    if triggers[i][a]==None: triggers[i][a]=0.0

        label_triggers.extend(triggers[i]) # skip those instances where triggers has None values
            

    sentence_embeddings=torch.stack(sentence_embeddings)
    return sentence_embeddings,None,label_triggers
  


def preprocess_model(dialogues,emotions,speakers_per_dialogue,triggers,tokenizer,max_len,Type='M1'):
    if Type=='M1' :
         return preprocess_for_ERC_Model1(dialogues,emotions,tokenizer,max_len)

    elif Type=='M2':
          return preprocess_for_ERC_Model2(dialogues,emotions)

    elif Type=='M3':
         return preprocess_for_EFR_Model3(dialogues,speakers_per_dialogue,triggers,tokenizer,max_len)

    elif Type=='M4':
          return preprocess_for_EFR_Model4(dialogues,emotions,speakers_per_dialogue,triggers)
    

def evaluator(gpu="F",checkpoint_dir="",Type='M1',batch_size=32,test_path='./test_file.json',model=None,max_len=192,criterion=nn.CrossEntropyLoss()):
    device = torch.device("cuda:3") if gpu == "T" else torch.device("cpu")
    
    if gpu=='F': checkpoint=torch.load(checkpoint_dir+f"last_checkpoint.pt",map_location=torch.device('cpu'))
    else: checkpoint=torch.load(checkpoint_dir+f"last_checkpoint.pt")
     
    model=model #initialize_model(Type=Type)
    model.load_state_dict(checkpoint['model_state_dict'])
    model.to(device)


    with open(test_path) as f:
        test_list = json.load(f)


    test_data=[]
    for item in test_list:
        episode = item['episode']
        speakers = item['speakers']
        emotions = item['emotions']
        utterances = item['utterances']
        triggers = item['triggers']
        test_data.append({'utterances':utterances,'emotions':emotions, 'triggers':triggers, 'speakers':speakers})



    data=test_data
    test_dataset=EmotionDataset(test_data,max_len=max_len,Type=Type)
    test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)
    predictions=[]
    true_labels=[]
    total_test_loss=0

    with torch.no_grad():
        for batch in test_dataloader:
            if Type=='M1':
                input_ids=batch[0].to(device)
                attention_masks=batch[1].to(device)
                labels=batch[2].to(device)
                outputs = model(input_ids=input_ids, attention_mask=attention_masks,labels=labels)
                loss = outputs.loss
                total_test_loss += loss.item()
                _, predicted = torch.max(outputs.logits, 1)
                predictions.extend(predicted.cpu().numpy())
                true_labels.extend(labels.cpu().numpy())

            if Type=='M2':
                sentence_embeddings=batch[0].to(device)
                labels=batch[1].to(device)
                outputs = model(sentence_embeddings)
                loss = criterion(outputs, labels)
                total_test_loss += loss.item()
                _, predicted = torch.max(outputs, 1)
                predictions.extend(predicted.cpu().numpy())
                true_labels.extend(labels.cpu().numpy())
                
            if Type=='M3':
                input_ids=batch[0].to(device)
                attention_masks=batch[1].to(device)
                labels=batch[2].to(device)
                outputs = model(input_ids=input_ids, attention_mask=attention_masks,labels=labels)
                loss = outputs.loss
                total_test_loss += loss.item()
                preds = outputs.logits.squeeze().detach().cpu().numpy()
                binary_preds = np.where(preds > 0.5, 1, 0)
                predictions.extend(binary_preds)
                true_labels.extend(labels.cpu().numpy())
                
            if Type=='M4':
                sentence_embeddings=batch[0].to(device)
                labels=batch[1].to(device)
                labels = torch.round(labels).long()
                outputs = model(sentence_embeddings)
                loss = criterion(outputs, labels)
                total_test_loss += loss.item()
                _, predicted = torch.max(outputs, 1)
                predictions.extend(predicted.cpu().numpy())
                true_labels.extend(labels.cpu().numpy())

    avg_test_loss = total_test_loss / len(true_labels)
    test_accuracy = accuracy_score(true_labels, predictions)
    test_f1 = f1_score(true_labels, predictions, average='weighted')

    print(f'Average Test Loss: {avg_test_loss:.4f}, Test accuracy: {test_accuracy:.4f}, Test F1 score: {test_f1:.4f}')

In [4]:
model_name = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(model_name)
bert_for_classification=BertForSequenceClassification.from_pretrained(model_name, num_labels=7)
bert_for_regression=BertForSequenceClassification.from_pretrained(model_name, num_labels=1)
bert_model=BertModel.from_pretrained(model_name)

class EmotionDataset(Dataset):
    def __init__(self, data,max_len=192,Type='M1'):
        self.data = data
        self.dialogues=[item['utterances'] for item in data] # list of dialogues  (a dialogue is a list of utterances)
        self.emotions = [emotion for item in data for emotion in item['emotions']]  # list of emotions (concatenated for all dialogues)
        self.emotion_per_dialogue= [item['emotions'] for item in data]
        self.speakers_per_dialogue= [item['speakers'] for item in data]
        self.triggers=[item['triggers'] for item in data] # list of triggers of each dialogue
        
        self.max_len=max_len
        self.max_emotion_pad_len=26
        self.Type=Type
        self.tokenizer=tokenizer
        self.bert_model=bert_model
        
        self.input_ids,self.attention_masks,self.labels=preprocess_model(self.dialogues,self.emotions,self.speakers_per_dialogue,self.triggers,self.tokenizer,self.max_len,Type)


    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        if self.Type=='M1' or self.Type=='M3':
            if (self.Type=='M3'): self.labels= torch.tensor(self.labels, dtype=torch.float) #self.labels.clone().detach().to(torch.float)
            return self.input_ids[idx], self.attention_masks[idx],self.labels[idx]

        if self.Type=='M2' or self.Type=='M4':
            if (self.Type=='M4'): self.labels= torch.tensor(self.labels, dtype=torch.float)
            return self.input_ids[idx],self.labels[idx]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
class GRUModel(nn.Module):
    def __init__(self, embedding_dim, hidden_dim, output_dim):
        super(GRUModel, self).__init__()
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.output_dim = output_dim
        self.gru = nn.GRU(input_size=embedding_dim, hidden_size=hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)
   
    def forward(self, x):
        x = x.unsqueeze(1)
        gru_output, _ = self.gru(x)
        output = self.fc(gru_output[:, -1, :])
        return output

In [6]:
import warnings
warnings.filterwarnings("ignore")

## Task 1

In [7]:
# Test has 60 samples
# 6 min on cpu 
evaluator(
    gpu='F',
    checkpoint_dir="./checkpoints/Task1_M1/",
    Type='M1',
    batch_size=32,
    test_path='./test_file.json',
    model=bert_for_classification, 
    max_len=192
)



evaluator(
    gpu='F',
    checkpoint_dir="./checkpoints/Task1_M2/",
    Type='M2',
    batch_size=32, 
    test_path='./test_file.json',
    model=GRUModel(embedding_dim=300, hidden_dim=128, output_dim=7)
)



Average Test Loss: 0.0017, Test accuracy: 0.9908, Test F1 score: 0.9908


Average Test Loss: 0.0442, Test accuracy: 0.4653, Test F1 score: 0.4072
Average Test Loss: 0.0040, Test accuracy: 0.8532, Test F1 score: 0.7856


Average Test Loss: 0.0149, Test accuracy: 0.8532, Test F1 score: 0.7856


## Task 2

In [8]:
evaluator(
    gpu='F',
    checkpoint_dir="./checkpoints/Task2_M3/",
    Type='M3',
    batch_size=32, 
    test_path='./test_file.json',
    model=bert_for_regression, 
    max_len=112
)


evaluator(
    gpu='F',
    checkpoint_dir="./checkpoints/Task2_M4/",
    Type='M4',
    batch_size=32, 
    test_path='./test_file.json',
    model=GRUModel(embedding_dim=300*3, hidden_dim=128, output_dim=2)
)

Average Test Loss: 0.0040, Test accuracy: 0.8532, Test F1 score: 0.7856


Average Test Loss: 0.0149, Test accuracy: 0.8532, Test F1 score: 0.7856
